# Analyzing proteomics data - GBA-PD vs. GBA-Ctrl

## 1) Use cohort browser to find Participant IDs for people that have proteomics data
According to data manifest and UKBB docs (https://biobank.ndph.ox.ac.uk/ukb/field.cgi?id=30900 ; https://dnanexus.gitbook.io/uk-biobank-rap/getting-started/data-structure):
- Add filter → Biological samples → Blood assays → Proteomics → Protein biomarkers → Number of proteins measured | Instance 0, Add cohort filter IS NOT NULL

52,995 subjects subjects have proteomics data (NOT null)

## 2) Count number of GBA-PD subjects with proteomics

In [ ]:
# Importing PD subjects
# (Health related outcomes -> First occurrences -> Nervous system disorders -> Source of report of G20 (parkinson's disease) | all except "Self-report only" and missing values)
import pandas as pd

PD_patients = pd.read_csv("/mnt/project/UKBB_participant_IDs_w_PD.csv")
PD_patients

In [ ]:
# Importing proteomics subjects
proteo_subjects = pd.read_csv("./UKBB_participant_IDs_w_proteomics.csv")
proteo_subjects

In [ ]:
# Download plink2
!wget https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip
!unzip plink2_linux_x86_64_latest.zip
!chmod +x plink2

!dx upload plink2 --destination ./

In [ ]:
# Defining a function to extract information (by Jeff Kim)
def extract_alternate_carriers_vectorized(raw_file, include_count=False):
    """
    Process PLINK2 raw format (--recode A) to extract participants carrying alternate alleles
    using vectorized operations for improved performance.
    
    Parameters:
    -----------
    raw_file : str
        Path to the PLINK2 raw format file
    include_count : bool, default=False
        If True, output will include a COUNT column with allele counts
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with columns:
        - IID: Sample identifier
        - VARID: Comma-separated list of variants with alternate alleles
        - COUNT: (Only if include_count=True) Comma-separated list of allele counts
                 corresponding to each variant in VARID
                 0 = Homozygous alternate, 1 = Heterozygous
    """
    # Read the raw file
    df = pd.read_csv(raw_file, sep = r'\s+')
    
    # Get the variant IDs (column names after the first 6 columns)
    variant_cols = df.columns[6:]
    
    # Pre-clean variant names (remove _REF part) - only do this operation once
    clean_variants = np.array([var.split('_')[0] for var in variant_cols])
    
    # Extract genotype data as numpy array for faster processing
    genotype_array = df[variant_cols].values
    
    # Get IIDs as numpy array
    iids = df['IID'].values
    
    # Initialize result dictionaries
    result_dict = {}
    count_dict = {}
    
    # Identify alternate allele carriers (value < 2)
    alt_allele_mask = genotype_array < 2
    
    # Process each sample
    for i in range(len(df)):
        # Find variants where this sample has alternate alleles
        alt_indices = np.where(alt_allele_mask[i])[0]
        
        # Only include samples with at least one alternate allele
        if len(alt_indices) > 0:
            # Get the variant IDs
            alternate_variants = clean_variants[alt_indices]
            
            # Join variant IDs
            result_dict[iids[i]] = ','.join(alternate_variants)
            
            if include_count:
                # Get the actual count values (0 or 1) for these variants
                count_values = genotype_array[i, alt_indices]
                # Convert to strings and join
                count_dict[iids[i]] = ','.join(map(str, count_values))
    
    # Convert the dictionaries to a DataFrame
    if include_count:
        result_df = pd.DataFrame({
            'IID': list(result_dict.keys()),
            'VARID': list(result_dict.values()),
            'COUNT': [count_dict[iid] for iid in result_dict.keys()]
        })
    else:
        result_df = pd.DataFrame(list(result_dict.items()), 
                               columns=['IID', 'VARID'])
    
    return result_df

In [ ]:
# Importing GBA carrier information (.raw file generated in "extracting_GBA_carriers_ok.ipynb")
import numpy as np

GBA1_carriers = extract_alternate_carriers_vectorized("/mnt/project/GBA1_risk_variants_raw.raw", include_count=True)
GBA1_E326K_carriers = GBA1_carriers[GBA1_carriers['VARID'].str.contains('1:155236376:C:T')]
GBA1_noE326K_carriers = GBA1_carriers[GBA1_carriers['VARID'] != ('1:155236376:C:T')]

In [ ]:
# Find GBA-PD cases through ID overlaps
GBA1_PD_carriers = GBA1_carriers[GBA1_carriers['IID'].isin(PD_patients['Participant ID'])]
GBA1_PD_carriers

In [ ]:
GBA1_PD_E326K_carriers = GBA1_E326K_carriers[GBA1_E326K_carriers['IID'].isin(PD_patients['Participant ID'])]
GBA1_PD_E326K_carriers

In [ ]:
GBA1_PD_noE326K_carriers = GBA1_noE326K_carriers[GBA1_noE326K_carriers['IID'].isin(PD_patients['Participant ID'])]
GBA1_PD_noE326K_carriers

In [ ]:
# Find GBA-PD carriers with proteomics
GBA1_PD_proteo = GBA1_PD_carriers[GBA1_PD_carriers['IID'].isin(proteo_subjects['column'])]
GBA1_PD_proteo

## 3) Count number of GBA-Ctrl subjects with proteomics

In [ ]:
# First, use cohort browser to get trustworthy list of "no PD" subjects
# (Health related outcomes -> First occurrences -> Nervous system disorders -> Source of report of G20 (parkinson's disease) | none with PD or missing values;
# No family history)

# Jeff already created participant lists of family history and they're inside "participant_list"
PD_parent_history = pd.read_csv("/mnt/project/participant_list/UKB_has_PD_parent.csv")
PD_parent_history.shape[0]

In [ ]:
PD_sibling_history = pd.read_csv("/mnt/project/participant_list/UKB_has_PD_sibling.csv")
PD_sibling_history.shape[0]

In [ ]:
PD_history = PD_parent_history['column'] + PD_sibling_history['column']
PD_history.shape[0] # is a list

In [ ]:
GBA1_carriers.shape[0]

In [ ]:
GBA1_Ctrl_carriers = GBA1_carriers[~GBA1_carriers['IID'].isin(PD_patients['Participant ID'])]
GBA1_Ctrl_carriers.shape[0]

In [ ]:
NoPD_subjects = pd.read_csv("./UKBB_participant_IDs_without_PD.csv")
NoPD_subjects.shape[0]

In [ ]:
NoPD_NoHistory = NoPD_subjects[~NoPD_subjects['column'].isin(PD_history)]
NoPD_NoHistory.shape[0]

In [ ]:
# Finally counting GBA-Ctrl and GBA-Ctrl with proteomics
GBA1_Ctrl_carriers = GBA1_carriers[GBA1_carriers['IID'].isin(NoPD_NoHistory['column'])]
GBA1_Ctrl_carriers

In [ ]:
GBA1_Ctrl_E326K_carriers = GBA1_E326K_carriers[GBA1_E326K_carriers['IID'].isin(NoPD_NoHistory['column'])]
GBA1_Ctrl_E326K_carriers.shape[0]

In [ ]:
GBA1_Ctrl_noE326K_carriers = GBA1_noE326K_carriers[GBA1_noE326K_carriers['IID'].isin(NoPD_NoHistory['column'])]
GBA1_Ctrl_noE326K_carriers.shape[0]

In [ ]:
GBA1_Ctrl_proteo = GBA1_Ctrl_carriers[GBA1_Ctrl_carriers['IID'].isin(proteo_subjects['column'])]
GBA1_Ctrl_proteo

In [ ]:
# Save all relevant files into permanent storage
GBA1_PD_proteo.to_csv("GBA1_PD_proteo.txt", sep="\t", index=False)
GBA1_Ctrl_proteo.to_csv("GBA1_Ctrl_proteo.txt", sep="\t", index=False)

!dx upload UKBB_participant* --destination ./
!dx upload GBA1_* --destination ./

## 4) Find and extract proteomics data
Useful GitHub repositories: https://github.com/dnanexus/UKB_RAP/blob/main/proteomics/ ; https://github.com/UK-Biobank/UKB-RAP-Notebooks-Access/blob/main/

In [ ]:
!dx ls Bulk/

In [ ]:
# Import packages
# dxpy allows python to interact with the platform storage
import dxpy
import pandas as pd
import subprocess
import glob
import os

In [ ]:
output_dir = "/proteomics_results/"

In [ ]:
# Automatically discover dispensed dataset ID
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)
dispensed_dataset_id = dispensed_dataset["id"]

In [ ]:
dispensed_dataset

In [ ]:
# Get project ID
project_id = dxpy.find_one_project()["id"]
project_id

In [ ]:
dataset = (":").join([project_id, dispensed_dataset_id])
dataset

In [ ]:
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

In [ ]:
# Get field names
path = os.getcwd()

data_dict_csv = glob.glob(os.path.join(path, "*.data_dictionary.csv"))[0]
data_dict_df = pd.read_csv(data_dict_csv)
data_dict_df.head()

In [ ]:
field_names = list(
    data_dict_df.loc[data_dict_df["entity"] == "olink_instance_0", "name"].values
)
print(len(field_names))

In [ ]:
field_names[1:20]

In [ ]:
# Look for CTS proteins
matches = [item for item in field_names if "cts".lower() in item.lower()]
matches

In [ ]:
## Extract abundance data itself
# For that, you need to open a Spark instance (basically, open a new JupyterLab instance and select a Spark cluster).

# An automatic Jupyter notebook will be created (Spark_notebook.py), which will already have some sample code to help you with the extraction, but you can also use code from the following notebooks:
# https://github.com/UK-Biobank/UKB-RAP-Notebooks-Access/blob/main/JupyterNotebook_R/A108_Constructing-the-Olink-dataset_R.ipynb
# https://github.com/dnanexus/UKB_RAP/blob/main/proteomics/0_extract_phenotype_protein_data.ipynb

# After extracting the data, don't forget to save into your workspace and load it here.

## 5. Basic preprocessing steps
1. (pre-processing) Filter samples based on metadata
2. (exploration) Examine the distribution of expression per protein
3. (exploration) PCA visualization of the data to determine if there are any outliers
4. (pre-processing) (if needed) Normalization

Sample notebook: https://github.com/dnanexus/UKB_RAP/blob/main/proteomics/protein_DE_analysis/1_preprocess_explore_data.ipynb

In [ ]:
# Rescuing patient information
import os
import pandas as pd
import numpy as np

GBA1_PD_proteo = pd.read_csv("/mnt/project/GBA1_PD_proteo.txt", sep="\t")
GBA1_Ctrl_proteo = pd.read_csv("/mnt/project/GBA1_Ctrl_proteo.txt", sep="\t")

GBA1_PD_proteo.head()

In [ ]:
# Rescuing protein abundances
proteomics = pd.read_csv("complete_proteomics_df.txt", sep="\t") # also in DNAnexus:proteomics_results
proteomics.head(5)

In [ ]:
# Look for CTS proteins
proteomics.columns = proteomics.columns.str.replace("olink_instance_0.", "", case=False)

matches2 = [col for col in proteomics.columns if col.lower().startswith("cts")]
matches2

In [ ]:
GBA1_PD_abund = proteomics[proteomics['eid'].isin(GBA1_PD_proteo['IID'])]
GBA1_Ctrl_abund = proteomics[proteomics['eid'].isin(GBA1_Ctrl_proteo['IID'])]

In [ ]:
len(GBA1_PD_abund)

In [ ]:
len(GBA1_Ctrl_abund)

In [ ]:
GBA1_PD_abund_cts = GBA1_PD_abund[ ["eid"] + matches2 ]
GBA1_PD_abund_cts.head()

In [ ]:
GBA1_Ctrl_abund_cts = GBA1_Ctrl_abund[ ["eid"] + matches2 ]
GBA1_Ctrl_abund_cts.head()

In [ ]:
# Check data distribution
GBA1_PD_abund_cts["ctsb"].hist()

In [ ]:
GBA1_Ctrl_abund_cts["ctsb"].hist()

In [ ]:
# PCA plot
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns

df = GBA1_PD_abund_cts.iloc[:, 1:].dropna()
numeric_df = df.select_dtypes(include='number')
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

pca = PCA(n_components=2)  # 2 components for 2D plot
pca_result = pca.fit_transform(scaled_data)

# Convert to a dataframe for plotting
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

plt.figure(figsize=(8,6))
plt.scatter(pca_df['PC1'], pca_df['PC2'])
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.title("PCA Plot")
plt.show()

In [ ]:
df = GBA1_Ctrl_abund_cts.iloc[:, 1:].dropna()
numeric_df = df.select_dtypes(include='number')
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_df)

pca = PCA(n_components=2)  # 2 components for 2D plot
pca_result = pca.fit_transform(scaled_data)

# Convert to a dataframe for plotting
pca_df = pd.DataFrame(pca_result, columns=['PC1', 'PC2'])

plt.figure(figsize=(8,6))
plt.scatter(pca_df['PC1'], pca_df['PC2'])
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.title("PCA Plot")
plt.show()

## 6. Basic "differential expression" analysis
Since our goal is just to have a quick look at the protein expression of cathepsins between GBA-PD and GBA-Ctrl subjects, we will run simple t-tests not adjusted for multiple comparisons nor adjusted for covariates.
We also did not impute the data, so we only used non-NA values.

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind
import numpy as np

# Example dataframes
# cases_df and controls_df have the same columns: first column is 'subject_id', others are proteins
# Drop subject ID for analysis
case_proteins = GBA1_PD_abund_cts.iloc[:, 1:]
control_proteins = GBA1_Ctrl_abund_cts.iloc[:, 1:]

# Prepare output dataframe
results = pd.DataFrame(columns=["protein", "mean_case", "mean_control", "log2FC", "p_value"])

# Loop through each protein
for protein in case_proteins.columns:
    case_values = case_proteins[protein].dropna()
    control_values = control_proteins[protein].dropna()
    
    # t-test
    t_stat, p_val = ttest_ind(case_values, control_values, equal_var=False)
    
    # Means
    mean_case = case_values.mean()
    mean_control = control_values.mean()
    
    # log2 fold change
    # Add small pseudocount if any mean is zero to avoid division by zero
    #log2fc = np.log2((mean_case + 1e-9) / (mean_control + 1e-9))
    log2fc = mean_case - mean_control # data seems to be already log2 transformed
    
    # Append to results
    results = pd.concat([results, pd.DataFrame({
        "protein": [protein],
        "mean_case": [mean_case],
        "mean_control": [mean_control],
        "log2FC": [log2fc],
        "p_value": [p_val]
    })], ignore_index=True)

# Optional: sort by p-value
results = results.sort_values("p_value")

In [ ]:
# Show results
results

In [ ]:
# Plot boxplots of selected proteins
selected_proteins = ["ctsv", "ctsz"]

# Melt dataframes to long format
cases_long = GBA1_PD_abund_cts
controls_long = GBA1_Ctrl_abund_cts

cases_long = cases_long.melt(id_vars="eid", value_vars=selected_proteins, 
                            var_name="protein", value_name="abundance")
cases_long["group"] = "case"

controls_long = controls_long.melt(id_vars="eid", value_vars=selected_proteins, 
                                 var_name="protein", value_name="abundance")
controls_long["group"] = "control"

# Combine
plot_df = pd.concat([cases_long, controls_long], ignore_index=True)

# Replace inf/-inf with NaN
plot_df.replace([np.inf, -np.inf], np.nan, inplace=True)

# Drop rows with NaN
plot_df.dropna(inplace=True)

In [ ]:
import seaborn as sns

plt.figure(figsize=(6,10))
sns.boxplot(x="protein", y="abundance", hue="group", data=plot_df, palette="Set2",
            hue_order=["control", "case"])
sns.stripplot(x="protein", y="abundance", hue="group", data=plot_df, 
              color="black", alpha=0.5, dodge=True, jitter=True,
              hue_order=["control", "case"])

plt.title("Protein abundance: Case vs Control")
plt.ylabel("Abundance")
plt.xlabel("Protein")
plt.legend(title="Group")
plt.show()

In [ ]:
# Upload final files
results.to_csv("t-test_CTS_proteins_GBA-PD_vs_GBA-Ctrl.txt", sep="\t", index=False)
!dx upload t-test_CTS_proteins_GBA-PD_vs_GBA-Ctrl.txt --destination proteomics_results/

plt.savefig("CTS_boxplots_GBA-PD_vs_GBA-Ctrl.pdf", format="pdf", bbox_inches="tight")  # tight trims extra whitespace
!dx upload CTS_boxplots_GBA-PD_vs_GBA-Ctrl.pdf --destination proteomics_results/